In [18]:
import plotly.graph_objects as go
import plotly.express as px
import seaborn as sns
import os
import pickle
import numpy as np
import pandas as pd
import sklearn
from sklearn.preprocessing import OneHotEncoder
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import KFold 
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler,scale,MinMaxScaler
from sklearn.metrics import mean_squared_error

from dataprepare.dataprep import *
from model.knn import *
from predictor.batterypredict import*

In [6]:
# import battery dataset, may have to change path
df_battery = pd.read_csv('../data/Battery_Dataset.csv')

In [31]:
user_in = ['graphite', 'LFP', 50, 25, 1.0]
pulled_data = user_report(df_battery, user_in, report)

# Below is visualization testing #

In [32]:
N = 70

fig = go.Figure(data=[go.Mesh3d(x=(70*np.random.randn(N)),
                   y=(55*np.random.randn(N)),
                   z=(40*np.random.randn(N)),
                   opacity=0.5,
                   color='rgba(244,22,100,0.6)'
                  )])

fig.update_layout(
    scene = dict(
        xaxis = dict(nticks=4, range=[-100,100],),
                     yaxis = dict(nticks=4, range=[-50,100],),
                     zaxis = dict(nticks=4, range=[-100,100],),),
    width=700,
    margin=dict(r=20, l=10, b=10, t=10))

fig.show()

In [33]:
x = [1,1,1] # input, strings (ex: 'NMC')
y = [50,100,150] # input
z = [13, 69, 21] # output

fig = go.Figure(data=go.Scatter3d(
    x=x, y=y, z=z,
    marker=dict(
        size=10,
        color = x
    ),
    line=dict(
        color=x,
        width=3,
        dash='dash'
    ),
    name='battery 1', text = 'battery 1'
))

fig.update_layout(title ={'text' :'help me', 
                          'x' : 0.5},
                 scene = dict(
                     xaxis = dict(
                         nticks = 3,
                         ticktext = ['NMC', 'NFP', 'LCA'],
                         tickvals = [0, 1, 2],
                     title = 'help'))
                 )
fig.show()

In [34]:
pulled_data.columns

Index(['anode', 'cathode', 'electrolyte', 'Cycle', 'temperature',
       'discharge_crate', 'Charge_Capacity (Ah)', 'Discharge_Capacity (Ah)',
       'Charge_Energy (Wh)', 'Discharge_Energy (Wh)',
       'Coulombic_Efficiency (%)', 'Energy_Efficiency (%)'],
      dtype='object')

In [35]:
# first 10 points are A123
elec_a123 = pulled_data.iloc[0:10, :]

# next 10 are Pan
elec_pan = pulled_data.iloc[10:20, :]

# last ten are LGC
elec_LGC = pulled_data.iloc[20:30, :]

fig = go.Figure(data=go.Scatter3d(
    x=elec_a123['electrolyte'], 
    y=elec_a123['Cycle'], 
    z=elec_a123['Charge_Capacity (Ah)'],
    marker=dict(
        size=10,
        color = '#5271FF'
    ),
    line=dict(
        color='#5271FF',
        width=3,
        dash='dash'
    ),
    name='A123'
))


fig.add_trace(
    go.Scatter3d(
        x=elec_pan['electrolyte'],
        y=elec_pan['Cycle'],
        z=elec_pan['Charge_Capacity (Ah)'],
        marker=dict(
            size=10,
            color = '#FF5757'
        ),
        line=dict(
            color='#FF5757',
            width=3,
            dash='dash'
        ),
        name='Panasonic'
    )
)


fig.add_trace(
    go.Scatter3d(
        x=elec_LGC['electrolyte'], 
        y=elec_LGC['Cycle'],
        z=elec_LGC['Charge_Capacity (Ah)'],
        marker=dict(
            size=10,
            color = '#FFBD59'
            #color = '#545454'
        ),
        line=dict(
            color = '#FFBD59',
            #color='#545454',
            width=3,
            dash='dash'
        ),
        name='LG Chem'
    )
)


fig.update_layout(title ={'text' :user_in[1] + ' Cathode Charge Capacity', 
                          'x' : 0.5},
                  autosize=False,
                  width=800,
                  height=600,
                  font = dict(size = 15),
                  scene = dict(
                      xaxis = dict(
                          nticks = 3,
                          title = 'Electrolyte'),
                      yaxis = dict(
                          title = 'Cycles'),
                      zaxis = dict(
                          title = 'Charge Capacity [Ah]')
                      )
                  )

fig.update_yaxes(automargin=True)

fig.show()

In [36]:
fig1 = go.Figure(data=go.Scatter(x=elec_a123['Cycle'], 
                                 y=elec_a123['Coulombic_Efficiency (%)'],
                                 marker=dict(
                                 size=10,
                                 color = '#5271FF'
                                ),
                                line=dict(
                                    color='#5271FF',
                                    width=3,
                                    dash='dash'
                                ),
                                name='A123')
                )

fig1.add_trace(go.Scatter(x=elec_pan['Cycle'], 
                                 y=elec_pan['Coulombic_Efficiency (%)'],
                                 marker=dict(
                                     size=15,
                                     color = '#FF5757',
                                     symbol = 'triangle-up'
                                 ),
                                 line=dict(
                                     color='#FF5757',
                                     width=3,
                                     dash='dash'
                                 ),
                                 name='Panasonic'
                         )
              )

fig1.add_trace(go.Scatter(x=elec_LGC['Cycle'], 
                          y=elec_LGC['Coulombic_Efficiency (%)'],
                          marker=dict(
                              size=10,
                              color = '#FFBD59',
                              #color = '#545454',
                              symbol = 'square'
                          ),
                          line=dict(
                              color = '#FFBD59',
                              #color='#545454',
                              width=3,
                              dash='dash'
                          ),
                          name='LG Chem'
                         )
              )

fig1.update_layout(title ={'text' :user_in[1] + ' Coulombic Efficiency', 
                          'x' : 0.45},
                   font = dict(size = 15),
                   xaxis = dict(
                       title = 'Number of Cycles'),
                   yaxis = dict(
                       title = 'CE (%)')
                  )
fig1.show()

In [37]:
x = [0,0,0,1,1,1,2,2,2] # input, strings (ex: 'NMC')
y = [50,100,150, 50,100,150, 50, 100, 150] # input
z = [13, 69, 21, 5, 25,50, 60, 40, 20] # output

df = pd.DataFrame([x,y,z]).T

# to avoid the connected lines between data
# sort data
# if = 1, lca, etc

fig = go.Figure(data=go.Scatter3d(
    x=df[0], y=df[1], z=df[2],
    marker=dict(
        size=10,
        color = x
    ),
    line=dict(
        color=x,
        width=3,
        dash='dash'
    ),
    name='battery 1', text = 'battery 1'
))

fig.update_layout(title ={'text' :'help me', 
                          'x' : 0.5},
                 scene = dict(
                     xaxis = dict(
                         nticks = 3,
                         ticktext = ['NMC', 'NFP', 'LCA'],
                         tickvals = [0, 1, 2],
                     title = 'help'))
                 )
fig.show()

df.head()

,0,1,2
0,0,50,13
1,0,100,69
2,0,150,21
3,1,50,5
4,1,100,25


In [38]:
x = [1,1,1] # input, strings (ex: 'NMC')
y = [50,100,150] # input
z = [13, 69, 21] # output

fig = px.scatter_3d(x=x, y=y, z=z,
                    color=x, 
                    labels = {'x': 'Battery System', 'y': 'Cycle Number',
                              'z': 'Output'})

fig.update_layout(title ={'text' :'help me', 
                          'x' : 0.5},
                 scene = dict(
                     xaxis = dict(
                         nticks = 3,
                         ticktext = ['NMC', 'NFP', 'LCA'],
                         tickvals = [0, 1, 2]))
                 )

fig.show()

In [39]:
x = [1,1,1] # input, strings (ex: 'NMC')
y = [50,100,150] # input
z = [13, 69, 21] # output

fig = go.Figure(data=px.line_3d(
    x=x, y=y, z=z,
    title='spinny boi',                    
    labels = {'x': 'Battery System', 'y': 'Cycle Number', 'z': 'Output'},
    markers = True
     ))

fig.update_layout(title = {'text' :'help me', 'x' : 0.5},
                 scene = dict(
                     xaxis = dict(
                         nticks = 3,
                         ticktext = ['NMC', 'NFP', 'LCA'],
                         tickvals = [0, 1, 2]))
                 )

fig.show()